## Startup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.api import VAR
from statsmodels.tools.eval_measures import aic, bic

import os
import pickle

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
np.random.seed(42)

In [4]:
dataroute=os.path.join("..",  "data")
dumproute=os.path.join("..",  "dump")
resultsroute=os.path.join("..",  "results")

## Data Retrieval

In [5]:
start='2013-01-01'
end="2023-06-01"

name=f'processed_dataset_{start}_{end}.pickle'
filename=os.path.join(dataroute, name)
with open(filename, 'rb') as handle:
    data=pickle.load(handle)
    
name=f'finaldf_dataset_{start}_{end}.pickle'
filename=os.path.join(dataroute, name)
with open(filename, 'rb') as handle:
    df=pickle.load(handle)

In [6]:
with open(os.path.join(dumproute, "tickerlist.pickle"), 'rb') as f:
    tickerlist=pickle.load(f)
tickerlist

['^MERV',
 'GGAL',
 'GGAL.BA',
 'YPF',
 'YPFD.BA',
 'EDN',
 'EDN.BA',
 'BMA',
 'BMA.BA']

In [7]:
df.head(3)

,^MERV_rets,^MERV_log_rets,^MERV_gk_vol,GGAL_rets,GGAL_log_rets,GGAL_gk_vol,GGAL.BA_rets,GGAL.BA_log_rets,GGAL.BA_gk_vol,YPF_rets,...,BMA_gk_vol,BMA.BA_rets,BMA.BA_log_rets,BMA.BA_gk_vol,USD_rets,USD_log_rets,USD_gk_vol,USD_^MERV_rets,USD_^MERV_log_rets,USD_^MERV_gk_vol
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-03,0.007552,0.007524,0.000129,-0.012748,-0.012830,0.001228,0.010616,0.010560,0.000677,-0.004016,...,0.000557,-0.007812,-0.007843,0.000857,0.011942,0.011871,0.000020,-0.002186,-0.002188,0.000129
2013-01-04,0.007092,0.007067,0.000158,-0.010043,-0.010094,0.000554,-0.006303,-0.006323,0.000208,-0.000672,...,0.000695,-0.027559,-0.027946,0.000692,0.021496,0.021269,0.000228,-0.008133,-0.008167,0.000158
2013-01-07,-0.001035,-0.001035,0.000022,-0.014493,-0.014599,0.000517,0.002114,0.002112,0.000063,0.005380,...,0.000862,0.012146,0.012073,0.000494,-0.004061,-0.004069,0.000017,-0.007749,-0.007779,0.000022


# VAR Training

In [8]:
log_rets_list=[column for column in df.columns if column.endswith("log_rets")]
vol_list=[column for column in df.columns if column.endswith("vol")]
simple_rets_list=[column for column in df.columns if (column.endswith("log_rets")) and (column not in log_rets_list)]

In [9]:
emptydf=pd.DataFrame(columns=["AIC", "BIC"], index=range(1,11))
results_dict_df={stock:emptydf for stock in tickerlist}

In [10]:
aic_best_model={stock:None for stock in tickerlist}
bic_best_model={stock:None for stock in tickerlist}

aic_best_residuals={stock:None for stock in tickerlist}
bic_best_residuals={stock:None for stock in tickerlist}

In [11]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    stock_data = df[columns]
    
    for lag in range(1, 11):
        model = VAR(stock_data)
        results = model.fit(lag)

        results_dict_df[stock].loc[lag, "AIC"]=results.aic
        results_dict_df[stock].loc[lag, "BIC"]=results.bic

    best_aic_lag=results_dict_df[stock]["AIC"].astype(float).idxmin()
    best_bic_lag=results_dict_df[stock]["BIC"].astype(float).idxmin()

    aic_best_model[stock]=VAR(stock_data).fit(best_aic_lag)
    bic_best_model[stock]=VAR(stock_data).fit(best_bic_lag)

    aic_best_residuals[stock]=aic_best_model[stock].resid
    bic_best_residuals[stock]=bic_best_model[stock].resid

In [15]:
with open(os.path.join(resultsroute, "VAR_univ_aic_bestmodels.pickle"), "wb") as output_file:
    pickle.dump(aic_best_model, output_file)

with open(os.path.join(resultsroute, "VAR_univ_bic_bestmodels.pickle"), "wb") as output_file:
    pickle.dump(bic_best_model, output_file)

Los modelos sirven los residuos NO!
https://github.com/alfsn/regime-switching-hmm/issues/27

In [16]:
with open(os.path.join(resultsroute, "VAR_univ_aic_residuals.pickle"), "wb") as output_file:
    pickle.dump(aic_best_residuals, output_file)

with open(os.path.join(resultsroute, "VAR_univ_bic_residuals.pickle"), "wb") as output_file:
    pickle.dump(bic_best_residuals, output_file)

# with USD

In [29]:
emptydf=pd.DataFrame(columns=["AIC", "BIC"], index=range(1,11))
results_dict_df={stock:emptydf for stock in tickerlist}

In [30]:
aic_best_model={stock:None for stock in tickerlist}
bic_best_model={stock:None for stock in tickerlist}

aic_best_residuals={stock:None for stock in tickerlist}
bic_best_residuals={stock:None for stock in tickerlist}

In [31]:
for stock in tickerlist:
    columns = ['USD_log_rets', 'USD_gk_vol', f'{stock}_log_rets', f'{stock}_gk_vol']
    stock_data = df[columns]
    
    for lag in range(1, 11):
        model = VAR(stock_data)
        results = model.fit(lag)

        results_dict_df[stock].loc[lag, "AIC"]=results.aic
        results_dict_df[stock].loc[lag, "BIC"]=results.bic

    best_aic_lag=results_dict_df[stock]["AIC"].astype(float).idxmin()
    best_bic_lag=results_dict_df[stock]["BIC"].astype(float).idxmin()

    aic_best_model[stock]=VAR(stock_data).fit(best_aic_lag)
    bic_best_model[stock]=VAR(stock_data).fit(best_bic_lag)

    aic_best_residuals[stock]=aic_best_model[stock].resid
    bic_best_residuals[stock]=bic_best_model[stock].resid

In [32]:
with open(os.path.join(resultsroute, "VAR_multiv_aic_bestmodels.pickle"), "wb") as output_file:
    pickle.dump(aic_best_model, output_file)

with open(os.path.join(resultsroute, "VAR_multiv_bic_bestmodels.pickle"), "wb") as output_file:
    pickle.dump(bic_best_model, output_file)

Los modelos sirven los residuos NO!
https://github.com/alfsn/regime-switching-hmm/issues/27

In [33]:
with open(os.path.join(resultsroute, "VAR_multiv_aic_residuals.pickle"), "wb") as output_file:
    pickle.dump(aic_best_residuals, output_file)

with open(os.path.join(resultsroute, "VAR_multiv_bic_residuals.pickle"), "wb") as output_file:
    pickle.dump(bic_best_residuals, output_file)